In [1]:
import os
import requests
import pandas as pd
import json

from pathlib import Path
import csv

from dotenv import load_dotenv
load_dotenv()

import pandas_datareader as web
import datetime as dt

%matplotlib inline

In [2]:
start = dt.datetime(2014,8,1)
end = dt.datetime.now()

btc_date = dt.datetime(2014,8,1)

iex_api_key = os.getenv("IEX_API_KEY")
iex_test_api_key = os.getenv("IEX_TEST_API_KEY")

base_url = 'https://cloud.iexapis.com/stable/'
sandbox_url = 'https://sandbox.iexapis.com/stable/'


token = os.environ.get('IEX_API_KEY')
test_token = os.environ.get('IEX_TEST_API_KEY')
params = {'token': token}
test_params = {'token': test_token}

test_resp = requests.get(base_url + 'status')
test_resp

<Response [200]>

In [3]:
def get_close(ticker):
    data = web.DataReader(ticker, 'yahoo', start, end)
    df = data[["Close"]]
    df.rename(columns={'Close':ticker},inplace=True)
    return df

def yahoo_10_year_data(tickers_list):
#     start = dt.datetime(2011,9,20)
#     end = dt.datetime.now()
    combined_df_f = pd.DataFrame()
    
    for i in tickers_list:
        if i == 0: 
            raw_df_1 = web.DataReader(i, 'yahoo', start, end)
            combined_df_f = raw_df_1[['Close']]
            combined_df_f.rename(columns={'Close':i},inplace=True)
            
        else:
            raw_df_2 = web.DataReader(i, 'yahoo', start, end)
            df_f = raw_df_2[['Close']]
            df_f.rename(columns={'Close':i},inplace=True)
            
            combined_df_f = pd.concat([combined_df_f,df_f], axis='columns')
            #combined_df_f = combined_df_f.merge(df_f, how='inner',right_index=True, left_index=True)


    return combined_df_f

def test_iex_apicall_get_close_data_73m(tickers_list):
    combined_df_f = pd.DataFrame()
    
    for i in tickers_list: 
        
        if i == 0: 
            resp_data = requests.get(sandbox_url+'stock/'+i+'/chart/75m?token='+test_token)
            combined_df_f = pd.DataFrame(resp_data.json())
            
            combined_df_f.rename(columns={'date':'Date'},inplace=True)
            combined_df_f['Date'] = pd.to_datetime(combined_df_f['Date'])
            combined_df_f = combined_df_f.set_index('Date')
            combined_df_f = combined_df_f[['close']]
            symbol = i 
            combined_df_f.rename(columns={'close':symbol},inplace=True)
            
        else: 
            resp_data = requests.get(sandbox_url+'stock/'+i+'/chart/75m?token='+test_token)
            resp_df = pd.DataFrame(resp_data.json())
            
            resp_df.rename(columns={'date':'Date'},inplace=True)
            resp_df['Date'] = pd.to_datetime(resp_df['Date'])
            resp_df = resp_df.set_index('Date')
            resp_df = resp_df[['close']]
            #symbol = data.iloc[0,4]
            symbol = i
            resp_df.rename(columns={'close':symbol},inplace=True)
            
            combined_df_f = pd.concat([combined_df_f,resp_df], axis='columns')
            
    return combined_df_f 

def real_iex_apicall_get_close_data_73m(tickers_list):
    combined_df_f = pd.DataFrame()
    
    for i in tickers_list: 
        
        if i == 0: 
            resp_data = requests.get(base_url+'stock/'+i+'/chart/75m?token='+token)
            combined_df_f = pd.DataFrame(resp_data.json())
            
            combined_df_f.rename(columns={'date':'Date'},inplace=True)
            combined_df_f['Date'] = pd.to_datetime(combined_df_f['Date'])
            combined_df_f = combined_df_f.set_index('Date')
            combined_df_f = combined_df_f[['close']]
            symbol = i 
            combined_df_f.rename(columns={'close':symbol},inplace=True)
            
        else: 
            resp_data = requests.get(base_url+'stock/'+i+'/chart/75m?token='+token)
            resp_df = pd.DataFrame(resp_data.json())
            
            resp_df.rename(columns={'date':'Date'},inplace=True)
            resp_df['Date'] = pd.to_datetime(resp_df['Date'])
            resp_df = resp_df.set_index('Date')
            resp_df = resp_df[['close']]
            #symbol = data.iloc[0,4]
            symbol = i
            resp_df.rename(columns={'close':symbol},inplace=True)
            
            combined_df_f = pd.concat([combined_df_f,resp_df], axis='columns')
            
    return combined_df_f

In [4]:
btc = get_close('BTC-USD')
eth = get_close('ETH-USD')
stock_list = ['GME','AMC','AMZN','WMT','NVDA','AMD','AAPL','MSFT','DIS','PYPL']
iex_stock_df = test_iex_apicall_get_close_data_73m(stock_list)

C:\Users\watso\anaconda3\envs\pyvizenv\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\watso\anaconda3\envs\pyvizenv\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [5]:
vus_df = get_close("VUS.TO")
crypto_df = btc.merge(eth, how='inner',right_index=True, left_index=True)
full_df = vus_df.merge(iex_stock_df, how='inner',right_index=True, left_index=True)
full_df = crypto_df.merge(full_df, how='inner',right_index=True, left_index=True)

C:\Users\watso\anaconda3\envs\pyvizenv\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [6]:
full_df

,BTC-USD,ETH-USD,VUS.TO,GME,AMC,AMZN,WMT,NVDA,AMD,AAPL,MSFT,DIS,PYPL
Date,,,,,,,,,,,,,
2015-08-06,279.584991,2.772120,42.209999,46.95,30.50,537.25,74.39,5.2873,2.20,29.1559,47.88,108.99,40.63
2015-08-07,260.997009,0.753325,41.970001,45.72,30.23,528.28,72.93,5.8420,2.13,30.0800,48.50,113.88,39.55
2015-08-10,270.385986,1.067860,42.599998,47.71,30.50,549.00,72.72,6.2290,1.97,31.3400,48.38,114.00,39.16
2015-08-11,266.376007,1.217440,42.220001,46.24,28.97,533.75,73.17,6.0780,1.90,29.6254,47.92,109.00,39.03
2015-08-12,264.079987,1.827670,42.270000,46.07,29.09,546.51,74.13,6.1500,2.00,29.9600,46.95,111.64,39.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-20,40693.675781,2764.431152,82.570000,192.80,40.62,3447.10,149.38,219.8500,106.49,148.3500,308.20,180.41,278.88
2021-09-21,43574.507812,3077.867920,82.589996,190.57,39.60,3348.36,149.87,214.2600,104.13,145.3300,296.60,178.21,275.14
2021-09-22,44895.097656,3155.523682,83.379997,193.53,41.10,3473.83,145.13,227.5500,104.55,153.0100,311.12,175.59,275.02


In [7]:
csvpath = Path('Resources/1810020501_databaseLoadingData.csv')
csv_data = pd.read_csv(csvpath)

In [8]:
land_and_house = csv_data[["REF_DATE","New housing price indexes","VALUE"]]
land_and_house['NEW_REF_DATE'] = pd.to_datetime((land_and_house['REF_DATE'].astype(str) + "-01"))
land_and_house.rename(columns={'NEW_REF_DATE':'Date'},inplace=True)
land_and_house.set_index("New housing price indexes",inplace=True)

total_land_and_house_value = land_and_house.loc["Total (house and land)"]
total_land_and_house_value.set_index("Date",inplace=True)
total_land_and_house_value.rename(columns={'VALUE':"Total land+house Value"},inplace=True)
total_land_and_house_value = total_land_and_house_value[['Total land+house Value']]

land_value = land_and_house.loc["Land only"]
land_value.set_index("Date",inplace=True)
land_value.rename(columns={'VALUE':"Land Value"},inplace=True)
land_value = land_value[['Land Value']]

house_value = land_and_house.loc["House only"]
house_value.set_index("Date",inplace=True)
house_value.rename(columns={'VALUE':"House Value"},inplace=True)
house_value = house_value[['House Value']]

C:\Users\watso\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\watso\anaconda3\envs\pyvizenv\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
combined_df = pd.concat([total_land_and_house_value,house_value,land_value], axis='columns')
only_house_values = combined_df[['House Value']]

In [10]:
iex_stock_df.to_csv('Resources/iex_stock_df.csv')
crypto_df.to_csv('Resources/crypto_df.csv')
full_df.to_csv('Resources/full_df.csv')
combined_df.to_csv('Resources/combined_df.csv')

In [11]:
iex_stock_df_date_index = pd.read_csv(
    Path("Resources/iex_stock_list.csv"),
    infer_datetime_format=True,
    parse_dates=True,
    index_col="Date",
)
house_price = pd.read_csv(
    Path("Resources/combined_df.csv"),
    infer_datetime_format=True,
    parse_dates=True,
    #index_col="Date",
)
house_value = house_price[["Date","House Value"]]
crypto_df = pd.read_csv(
    Path("Resources/crypto_df.csv"),
    infer_datetime_format=True,
    parse_dates=True,
    index_col="Date",
)

iex_stock_df_date_index = vus_df.merge(iex_stock_df_date_index, how='inner',right_index=True, left_index=True)
full_df = crypto_df.merge(iex_stock_df_date_index, how='inner',right_index=True, left_index=True)
full_df.reset_index(inplace=True)
new_df = full_df.iloc[::21,:]
new_df.reset_index(inplace=True)
new_df.drop(columns={"index","Date"},inplace=True)
final_df = pd.concat([house_value,new_df], axis="columns")
final_df.set_index("Date",inplace=True)
final_df.to_csv('Resources/final_df.csv')

C:\Users\watso\anaconda3\envs\pyvizenv\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [12]:
final_df

,House Value,BTC-USD,ETH-USD,VUS.TO,GME,AMC,AMZN,WMT,NVDA,AMD,AAPL,MSFT,DIS,PYPL
Date,,,,,,,,,,,,,,
2015-08-01,91.8,279.584991,2.772120,42.209999,46.65,30.09,534.96,75.94,5.3516,2.220,29.5414,47.59,108.99,39.62
2015-09-01,92.0,235.018997,1.338810,38.959999,43.29,27.95,523.00,67.02,5.5624,1.850,28.3139,42.75,104.93,35.29
2015-10-01,92.4,242.968994,0.609388,39.790001,44.16,26.83,563.36,67.83,6.6696,1.910,28.7236,46.85,106.76,32.76
2015-11-01,92.6,386.354004,0.895637,42.049999,48.64,27.27,663.18,60.25,7.1472,2.200,30.9800,56.88,117.00,37.86
2015-12-01,92.8,395.536011,0.811264,41.610001,33.30,23.90,694.04,60.60,8.5192,2.439,30.5100,56.13,117.71,35.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-01,118.3,37304.691406,2430.621338,78.820000,170.57,13.11,3253.92,145.12,148.4710,81.270,129.5400,248.27,173.44,264.06
2021-05-01,119.9,33723.027344,1989.736328,80.980003,225.60,59.55,3517.87,140.42,192.1264,85.420,134.4400,269.58,178.60,296.72
2021-06-01,120.7,34292.445312,2189.218750,83.430000,180.85,37.25,3665.96,145.69,196.9000,94.410,152.0900,295.86,179.32,309.34
